In [ ]:
#git clone https://github.com/lucidrains/iTransformer.git

In [22]:
# TODO which one?
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers
import config
print("Import succesfull")


Import succesfull


In [18]:
df = data_handling.load_eu_electricity()
df = df.drop('date', axis=1)

tensor_list = []

for column in df.columns:
	tensor = torch.tensor(df[column].values, dtype=torch.float32)
	tensor_list.append(tensor)

result_tensor = torch.stack(tensor_list)
result_tensor = result_tensor.transpose(0,1)

standardize_dict = None

#if standardize == True:
result_tensor, standardize_dict = helpers.custom_standardizer(result_tensor)

data_dict = {"train" : 0.7, "validation" : 0.1, "test" : 0.2}

for key, value in data_dict.items():
	data_dict[key] = value * result_tensor.shape(0)

for key, value in data_dict.items():
	data_dict[key] = result_tensor[0:value, :]

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 

In [24]:
window_size = 96
pred_length = (96, 192, 336, 720)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

electricity_dict = data_handling.load_electricity()
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(electricity_dict, window_size, pred_length)

Using device: cuda
Feature batch shape: torch.Size([32, 96, 348])


In [25]:
model_config = {
    'num_variates': electricity_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': 6,
    'dim': 512,
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': 32,
    'heads': 4,
    'attn_dropout': 0.1,
    'ff_mult': 4,
    'ff_dropout': 0.1,
    'num_mem_tokens': 4,
    'use_reversible_instance_norm': True,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

# defining all needed instances
model = iTransformer(**model_config).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])


In [26]:
def fast_eval(dataloader):
	model.eval()
	preds_dict = {
		96 : {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		},
		192 : {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		},
		336 : {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		},
		720: {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		}}

	with torch.no_grad():
		for input, target in tqdm(dataloader, desc=f"Epoch: Validating"):
			input = input.to(device)
		
			# targets are saved as dict, send each to device
			targets = (target[0].to(device), target[1].to(device), target[2].to(device), target[3].to(device))
			outputs = model(input)
		
		# for each prediciton length we calculate the metrics
		for target, output in zip(targets, outputs.values()):
			preds_dict[output.size(1)]["mse"] = preds_dict[output.size(1)]["mse"] + helpers.mean_squared_error(output, target)
			preds_dict[output.size(1)]["mae"] = preds_dict[output.size(1)]["mae"] + helpers.mean_absolute_error(output, target)
			preds_dict[output.size(1)]["smape"] = preds_dict[output.size(1)]["smape"] + helpers.symmetric_mean_absolute_percentage_error(output, target)
			preds_dict[output.size(1)]["p10"] = preds_dict[output.size(1)]["p10"] + helpers.percentile(output, target, 0.1)
			preds_dict[output.size(1)]["p50"] = preds_dict[output.size(1)]["p50"] + helpers.percentile(output, target, 0.5)
			preds_dict[output.size(1)]["p90"] = preds_dict[output.size(1)]["p90"] + helpers.percentile(output, target, 0.9)

	print(preds_dict[192].items())

	
	return preds_dict

In [27]:
def train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer):
	global_step = 0
	model.train()
	total_loss = 0
	
	for input, target in tqdm(dataloader_train, desc=f"Epoch: {epoch}"):
		optimizer.zero_grad()

		if len(target) == 4:
			loss = model(input.to(device), (target[0].to(device), target[1].to(device), target[2].to(device), target[3].to(device)))
		else:
			loss = model(input.to(device), target.to(device))

		loss.backward()
		optimizer.step()
		total_loss += loss.item()

		writer.add_scalar('train_loss', loss, global_step)
		writer.add_scalar('train_lr', optimizer.param_groups[0]['lr'], global_step)
		lr =  optimizer.param_groups[0]['lr']
		global_step+=1

	print(f'Epoch {epoch}, MSE-Loss: {total_loss / (len(dataloader_train) * 4)}, LR: {lr}')

	scheduler.step()
	writer.close()
	if epoch % 5 == 0:
		helpers.create_checkpoint(model, optimizer, scheduler, epoch, loss, global_step, "iTransfomrer_electricity")
	eval_metrics_dict = fast_eval(dataloader_validation)

epoch = 6

for epoch in range(1, epoch + 1):
	train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)


Epoch: 1: 100%|██████████| 131/131 [00:20<00:00,  6.29it/s]


Epoch 1, Loss: 0.25280982096686616, LR: 0.001


Epoch: Validating: 100%|██████████| 1/1 [00:00<00:00, 33.68it/s]


dict_items([('mse', tensor(0.2975, device='cuda:0')), ('mae', tensor(0.3579, device='cuda:0')), ('smape', tensor(118.9072, device='cuda:0')), ('p10', tensor(0.0390, device='cuda:0')), ('p50', tensor(0.2284, device='cuda:0')), ('p90', tensor(0.8304, device='cuda:0'))])


Epoch: 2: 100%|██████████| 131/131 [00:20<00:00,  6.30it/s]


...

Epoch: Validating: 100%|██████████| 1/1 [00:00<00:00, 36.11it/s]


dict_items([('mse', tensor(0.2513, device='cuda:0')), ('mae', tensor(0.3300, device='cuda:0')), ('smape', tensor(112.9122, device='cuda:0')), ('p10', tensor(0.0362, device='cuda:0')), ('p50', tensor(0.2134, device='cuda:0')), ('p90', tensor(0.7590, device='cuda:0'))])


Epoch: 5: 100%|██████████| 131/131 [00:20<00:00,  6.30it/s]

Epoch 5, Loss: 0.16709839364954535, LR: 0.001


KeyError: 'series_standardized'

In [33]:
def eval_function(valid_dataloader):
	try:
		model = iTransformer(**model_config)

	except UnboundLocalError:
		checkpoint = torch.load("/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/series_standardized/model_first_20_ids_full_epoch_10.pt")
		model.load_state_dict(checkpoint['model_state_dict'])
		optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
		scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
		scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
		writer = SummaryWriter(log_dir='logs')
		device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
		model.to(device)

	model.eval()
	global_loss_dict = {
					"mse" : 0,
					"mae" : 0,
					"p10" : 0,
					"p50" : 0,
					"p90" : 0
				}
	
	with torch.no_grad():
		for input, target in tqdm(valid_dataloader, desc=f"Epoch: Validating"):
			input = input.to(device)
			target = target.to(device)
			
			output = model(input)

			global_loss_dict = helpers.evaluate(global_loss_dict, output, target.to(device))

	for key, values in global_loss_dict.items():
		print(f"{key} loss: {values}")
		writer.add_scalar(key, value/len(valid_dataloader))
	writer.close()

In [34]:
eval_function(dataloader_validation)

ValueError: __len__() should return >= 0

In [35]:
eval_function(dataloader_test)

Epoch: Validating:   0%|          | 0/67 [00:00<?, ?it/s]


UnboundLocalError: local variable 'device' referenced before assignment